해상도를 두배 증가시킨 이미지를 최대한 겹치지않게 random으로 5개 crop하는 코드

In [ ]:
import json
import os
from PIL import Image, ImageDraw
from pycocotools.coco import COCO
from tqdm import tqdm
from datetime import datetime
import random
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
def random_crop(image, crop_size, existing_crops):
    img_width, img_height = image.size
    crop_width, crop_height = crop_size
    max_attempts = 100

    for _ in range(max_attempts):
        x_offset = random.randint(0, img_width - crop_width)
        y_offset = random.randint(0, img_height - crop_height)
        new_crop = (x_offset, y_offset, crop_width, crop_height)

        if not any(is_overlapping(new_crop, existing_crop) for existing_crop in existing_crops):
            return new_crop

    return new_crop

In [ ]:
def is_overlapping(crop1, crop2, iou_threshold=0.1):
    x1, y1, w1, h1 = crop1
    x2, y2, w2, h2 = crop2

    xa = max(x1, x2)
    ya = max(y1, y2)
    xb = min(x1 + w1, x2 + w2)
    yb = min(y1 + h1, y2 + h2)

    inter_width = max(0, xb - xa)
    inter_height = max(0, yb - ya)
    inter_area = inter_width * inter_height

    crop1_area = w1 * h1
    crop2_area = w2 * h2

    iou = inter_area / (crop1_area + crop2_area - inter_area)
    return iou > iou_threshold

In [ ]:
def update_annotations_for_subimage(annotations, subimg_info, img_id):
    updated_annotations = []
    x_offset, y_offset, subimg_width, subimg_height = subimg_info

    for ann in annotations:
        x, y, width, height = ann['bbox']

        x1 = max(x, x_offset)
        y1 = max(y, y_offset)
        x2 = min(x + width, x_offset + subimg_width)
        y2 = min(y + height, y_offset + subimg_height)

        if x1 < x2 and y1 < y2:
            new_x = x1 - x_offset
            new_y = y1 - y_offset
            new_width = x2 - x1
            new_height = y2 - y1

            if new_width > 0 and new_height > 0:
                updated_ann = ann.copy()
                updated_ann['bbox'] = [new_x, new_y, new_width, new_height]
                updated_ann['image_id'] = img_id
                
                if 'segmentation' in updated_ann:
                    updated_segmentation = []
                    for seg in updated_ann['segmentation']:
                        new_seg = []
                        for i in range(0, len(seg), 2):
                            new_x = seg[i] - x_offset
                            new_y = seg[i+1] - y_offset
                            if 0 <= new_x <= subimg_width and 0 <= new_y <= subimg_height:
                                new_seg.extend([new_x, new_y])
                        if len(new_seg) >= 6:
                            updated_segmentation.append(new_seg)
                    updated_ann['segmentation'] = updated_segmentation

                updated_annotations.append(updated_ann)
    
    return updated_annotations

In [ ]:
# Main processing
original_folder = '/data/ephemeral/home/dataset/'
sr_folder = '/data/ephemeral/home/EDSR-PyTorch/experiment/test/results-Demo/'
annotation_file = '/data/ephemeral/home/dataset/train.json'
subimgs_path = '/data/ephemeral/home/dataset/scale_x2_random'
updated_annotation_path = '/data/ephemeral/home/dataset/train_random_x2.json'

In [ ]:
with open(annotation_file, 'r') as file:
    data = json.load(file)

# Initialize new images and annotations lists
new_images = []
new_annotations = []
new_img_id = max([img['id'] for img in data['images']]) + 1

# Create COCO object and get image file list
coco = COCO(annotation_file)
image_files = os.listdir(os.path.join(sr_folder, 'train'))

crop_size = (1024, 1024)

In [ ]:
for idx in tqdm(image_files, desc="Processing images"):
    img_id = int(idx.split('_')[0])
    img_info = coco.loadImgs(img_id)[0]
    
    # Load SR image (2x size)
    I = Image.open(os.path.join(sr_folder, f"{img_info['file_name'].split('.')[0]}_x2_SR.png"))
    img_width, img_height = I.size

    # Get original annotations
    annIds = coco.getAnnIds(imgIds=img_id, iscrowd=None)
    original_anns = coco.loadAnns(annIds)

    # Adjust original annotation coordinates for SR image (2x)
    for ann in original_anns:
        ann['bbox'] = [coord * 2 for coord in ann['bbox']]

    existing_crops = []

    # Perform 5 random crops for each image
    for i in range(5):
        crop_info = random_crop(I, crop_size, existing_crops)
        updated_anns = update_annotations_for_subimage(original_anns, crop_info, new_img_id)

        # Save cropped image and add new annotations
        x_offset, y_offset, subimg_width, subimg_height = crop_info
        subimg = I.crop((x_offset, y_offset, x_offset + subimg_width, y_offset + subimg_height))

        subimg_filename = f"{img_info['file_name'].split('.')[0]}_random_crop_{i}_x2_SR.png"
        
        if updated_anns:
            new_img = {
                "width": subimg_width,
                "height": subimg_height,
                "file_name": subimg_filename,
                "date_captured": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                "id": new_img_id
            }
            new_images.append(new_img)
            new_annotations.extend(updated_anns)

            # Save subimg only if it contains bboxes
            subimg.save(os.path.join(subimgs_path, subimg_filename))
        
            new_img_id += 1

        existing_crops.append(crop_info)

print("Processing completed.")

In [ ]:
# Update annotation file
data['images'].extend(new_images)
data['annotations'].extend(new_annotations)

with open(updated_annotation_path, 'w') as file:
    json.dump(data, file, indent=2)

print("Annotation file updated successfully.")